In [1]:
from requests import get, post
from web3 import Web3, HTTPProvider
import web3
from icecream import ic
import numpy as np
import pandas as pd
import sys
from dotenv import load_dotenv
import os
import math
import json

In [2]:
# Add the directory containing the module to the Python path
module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

# Import functions from the module
from uniV3Pricing import get_account_data, get_uniswap_slot0, get_nft_positions_details, get_amounts_from_ticks

In [3]:
# Explicitly specify the path to .env file using forward slashes
load_dotenv(dotenv_path='.env')

# Create the Web3 provider using the Alchemy URL
w3 = Web3(HTTPProvider(os.getenv('BLOCKPI_BASE'))) # BASE

# Check connection
print(f"Web3 is connected: {w3.is_connected()}")


Web3 is connected: True


In [25]:
def sqrt_price_x96_to_price(sqrt_price_x96):
    Q96 = 0x1000000000000000000000000
    return (sqrt_price_x96 / Q96) ** 2


def price_to_tick(p):
    return math.floor(math.log(p, 1.0001))


def get_slot0_info(pool_address, w3):
    abi_uniswap = [
        {
            "inputs": [],
            "name": "slot0",
            "outputs": [
                {"internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
                {"internalType": "int24", "name": "tick", "type": "int24"},
                {"internalType": "uint16", "name": "observationIndex", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinality", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinalityNext", "type": "uint16"},
                {"internalType": "uint8", "name": "feeProtocol", "type": "uint8"},
                {"internalType": "bool", "name": "unlocked", "type": "bool"},
            ],
            "stateMutability": "view",
            "type": "function",
        }
    ]
    
    abi_aerodrome = [
        {
            "inputs": [],
            "name": "slot0",
            "outputs": [
                {"internalType": "uint160", "name": "sqrtPriceX96", "type": "uint160"},
                {"internalType": "int24", "name": "tick", "type": "int24"},
                {"internalType": "uint16", "name": "observationIndex", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinality", "type": "uint16"},
                {"internalType": "uint16", "name": "observationCardinalityNext", "type": "uint16"},
                {"internalType": "bool", "name": "unlocked", "type": "bool"},
            ],
            "stateMutability": "view",
            "type": "function",
        }
    ]
    
    contract_address = Web3.to_checksum_address(pool_address)
    
    # Try to use the Uniswap ABI
    try:
        contract = w3.eth.contract(address=contract_address, abi=abi_uniswap)
        result = contract.functions.slot0().call()
        return {
            "sqrtPriceX96": result[0],
            "tick": result[1]
        }
    except (web3.exceptions.ContractLogicError, web3.exceptions.BadFunctionCallOutput):
        # If Uniswap ABI fails, use the Aerodrome ABI
        try:
            contract = w3.eth.contract(address=contract_address, abi=abi_aerodrome)
            result = contract.functions.slot0().call()
            return {
                "sqrtPriceX96": result[0],
                "tick": result[1]
            }
        except (web3.exceptions.ContractLogicError, web3.exceptions.BadFunctionCallOutput):
            return None


In [26]:
def get_position_state_info(contract_address, nft_token_id, w3):
    abi = [
        {
        "inputs": [{"internalType": "uint256", "name": "position", "type": "uint256"}],
        "name": "positionState",
        "outputs": [
            {"internalType": "int24", "name": "tickLower", "type": "int24"},
            {"internalType": "int24", "name": "tickUpper", "type": "int24"},
            {"internalType": "uint128", "name": "liquidity", "type": "uint128"},
            {"internalType": "address", "name": "gauge", "type": "address"},
        ],
        "stateMutability": "view",
        "type": "function",
        }
    ]
    contract_address = Web3.to_checksum_address(contract_address)
    
    contract = w3.eth.contract(address=contract_address, abi=abi)
    result = contract.functions.positionState(nft_token_id).call()
    
    return {
            "tickLower": result[0],
            "tickUpper": result[1],
            "liquidity": result[2],
            "gauge": result[3]
        }

get_position_state_info("0x1dc7a0f5336f52724b650e39174cfcbbedd67bf1", 206932, w3)

{'tickLower': -85200,
 'tickUpper': -84000,
 'liquidity': 231414873946179291929,
 'gauge': '0x45F8b8eC9c92D09BA8495074436fD97073423041'}

In [39]:
from collections import defaultdict

# Function to add or update the dictionary
def add_or_update_dict(dictionary, key, value):
    dictionary[key] += value

def get_arcadia_account_nft_position(asset_data, w3):
    """
    Sample asset_data:
    [
        ["0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22", 
        "0x4200000000000000000000000000000000000006", 
        "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1"], 
        [0, 0, 105363], 
        [12289573077240346, 13102161708949451, 1]
    ]
    """
    POOL_NFT_MAPPINGS = [
        # Uniswap v3 Pools
        {
            "name": "wETH-USDC",
            "pool": "0xd0b53D9277642d899DF5C87A3966A349A798F224",
            "gauge": "",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "decimal0": 18,
            "decimal1": 6
        },
        {
            "name": "wETH-USDbC",
            "pool": "0x4C36388bE6F416A29C8d8Eee81C771cE6bE14B18",
            "gauge": "",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 18,
            "decimal1": 6
        },
        {
            "name": "wETH-wstETH",
            "pool": "0x20E068D76f9E90b90604500B84c7e19dCB923e7e",
            "gauge": "",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "cbETH-wETH",
            "pool": "0x10648BA41B8565907Cfa1496765fA4D95390aa0d",
            "gauge": "",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22",
            "token1": "0x4200000000000000000000000000000000000006",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "USDC-USDbC",
            "pool": "0x06959273E9A65433De71F5A452D529544E07dDD0",
            "gauge": "",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 6,
            "decimal1": 6
        },
        {
            "name": "DAI-USDbC",
            "pool": "0x22F9623817F152148B4E080E98Af66FBE9C5AdF8",
            "gauge": "",
            "nft": "0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1",
            "token0": "0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 18,
            "decimal1": 6
        },
        
        # Aerodrome Pools
        {
            "name": "wETH-wstETH",
            "pool": "0x861A2922bE165a5Bd41b1E482B49216b465e1B5F",
            "gauge": "0x2A1f7bf46bd975b5004b61c6040597E1B6117040",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "cbETH-wETH",
            "pool": "0x47cA96Ea59C13F72745928887f84C9F52C3D7348",
            "gauge": "0xF5550F8F0331B8CAA165046667f4E6628E9E3Aac",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22",
            "token1": "0x4200000000000000000000000000000000000006",
            "decimal0": 18,
            "decimal1": 18
        },
        {
            "name": "USDC-USDbC",
            "pool": "0x98c7A2338336d2d354663246F64676009c7bDa97",
            "gauge": "0x4a3E1294d7869567B387FC3d5e5Ccf14BE2Bbe0a",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
            "decimal0": 6,
            "decimal1": 6
        },
        {
            "name": "wETH-USDC",
            "pool": "0xb2cc224c1c9feE385f8ad6a55b4d94E92359DC59",
            "gauge": "0xF33a96b5932D9E9B9A0eDA447AbD8C9d48d2e0c8",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x4200000000000000000000000000000000000006",
            "token1": "0x833589fCD6eDb6E08f4c7C32D4f71b54bdA02913",
            "decimal0": 18,
            "decimal1": 6
        },
        {
            "name": "AERO-wstETH",
            "pool": "0x565aecF84b5d30a6E79a5CEf3f0dA0Fc4280dEBC",
            "gauge": "0x45F8b8eC9c92D09BA8495074436fD97073423041",
            "nft": "0x827922686190790b37229fd06084350E74485b72",
            "token0": "0x940181a94A35A4569E4529A3CDfB74e38FD98631",
            "token1": "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
            "decimal0": 18,
            "decimal1": 18
        }
    ]
    
    zero_indices = [index for index, value in enumerate(asset_data[1]) if value == 0]
    non_zero_indices = [index for index, value in enumerate(asset_data[1]) if value != 0]

    result = defaultdict(int)

    # Creating a lookup dictionary for quick access with normalized tokens
    pool_lookup = {}
    gauge_lookup = {}

    for pool in POOL_NFT_MAPPINGS:
        nft = pool["nft"].lower()
        token0 = pool["token0"].lower()
        token1 = pool["token1"].lower()
        gauge = pool["gauge"].lower() if pool["gauge"] else ""

        pool_lookup[(nft, token0, token1)] = pool
        if gauge:
            gauge_lookup[gauge] = pool

    for i in zero_indices:
        add_or_update_dict(result, asset_data[0][i], asset_data[2][i])
    
    for i in non_zero_indices:
        staked_slipstream = "0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1".lower()
        nft_contract = asset_data[0][i].lower()
        nft_token_id = asset_data[1][i]
        
        if nft_contract == staked_slipstream:
            staked_slipstream_data = get_position_state_info(nft_contract, nft_token_id, w3)
            ic(staked_slipstream_data)
            """
            {'tickLower': -85200,
            'tickUpper': -84000,
            'liquidity': 231414873946179291929,
            'gauge': '0x45F8b8eC9c92D09BA8495074436fD97073423041'}
            """
            matching_pool = gauge_lookup.get(staked_slipstream_data['gauge'].lower())
            ic(matching_pool)
            
            if not matching_pool:
                add_or_update_dict(result, nft_contract, 0)  # No matching pool found
                continue  # Skip to the next iteration
            
            lower_tick = staked_slipstream_data["tickLower"]
            upper_tick = staked_slipstream_data["tickUpper"]
            liquidity = staked_slipstream_data["liquidity"]
            
        else:
            nft_positions_details = get_nft_positions_details(nft_contract_address=nft_contract, w3=w3, token_id=nft_token_id)
            ic(nft_positions_details)
            
            if not nft_positions_details:
                add_or_update_dict(result, nft_contract, 0)  # Records the NFT if it does not belong to Uniswap
                continue  # Skip if nft_positions_details could not be fetched
            
            # Extracting and normalizing token0 and token1 from nft_positions_details
            token0 = nft_positions_details["token0"].lower()
            token1 = nft_positions_details["token1"].lower()

            # Finding the matching pool using the lookup dictionary
            matching_pool = pool_lookup.get((nft_contract, token0, token1))
            ic(matching_pool)
            
            if not matching_pool:
                add_or_update_dict(result, nft_contract, 0)  # No matching pool found
                continue  # Skip to the next iteration
            
            lower_tick = nft_positions_details["tickLower"]
            upper_tick = nft_positions_details["tickUpper"]
            liquidity = nft_positions_details["liquidity"]
            
        slot0 = get_slot0_info(pool_address=matching_pool["pool"], w3=w3)
        ic(slot0)
        if not slot0:
            continue  # Skip if slot0 details could not be fetched
        
        current_tick = slot0["tick"]
        
        amount0, amount1 = get_amounts_from_ticks(current_tick, lower_tick, upper_tick, liquidity)
        
        ic(amount0, amount1)
        
        add_or_update_dict(result, matching_pool["token0"], amount0)
        add_or_update_dict(result, matching_pool["token1"], amount1)
    
    return result

In [40]:
account_data_test = get_account_data("0x0c54555551956de18cd4cd3d0e5b45692aee8f01",w3)
print(json.dumps(account_data_test, indent=4))

position_distribution = get_arcadia_account_nft_position(account_data_test, w3)
print(json.dumps(position_distribution, indent=4))

ic| staked_slipstream_data: {'gauge': '0xF5550F8F0331B8CAA165046667f4E6628E9E3Aac',
                             'liquidity': 193388721924412646753,
                             'tickLower': 738,
                             'tickUpper': 741}
ic| matching_pool: {'decimal0': 18,
                    'decimal1': 18,
                    'gauge': '0xF5550F8F0331B8CAA165046667f4E6628E9E3Aac

[
    [
        "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452",
        "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22",
        "0x4200000000000000000000000000000000000006",
        "0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1",
        "0x1Dc7A0f5336F52724B650E39174cfcbbEdD67bF1"
    ],
    [
        0,
        0,
        0,
        198302,
        198301
    ],
    [
        39788340545748,
        75122175976705,
        153065914586519,
        1,
        1
    ]
]


',
                    'name': 'cbETH-wETH',
                    'nft': '0x827922686190790b37229fd06084350E74485b72',
                    'pool': '0x47cA96Ea59C13F72745928887f84C9F52C3D7348',
                    'token0': '0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22',
                    'token1': '0x4200000000000000000000000000000000000006'}
ic| slot0: {'sqrtPriceX96': 82208654178532228229029415521, 'tick': 738}
ic| amount0: 27953967328849011, amount1: 0
ic| staked_slipstream_data: {'gauge': '0x2A1f7bf46bd975b5004b61c6040597E1B6117040',
                             'liquidity': 185279128863115938566,
                             'tickLower': -1593,
                             'tickUpper': -1590}
ic| matching_pool: {'decimal0': 18,
                    'decimal1': 18,
                    'gauge': '0x2A1f7bf46bd975b5004b61c6040597E1B6117040',
                    'name': 'wETH-wstETH',
                    'nft': '0x827922686190790b37229fd06084350E74485b72',
                    'pool': '0x

{
    "0xc1CBa3fCea344f92D9239c08C0568f6F2F0ee452": 39788340545748,
    "0x2Ae3F1Ec7F1F5012CFEab0185bfc7aa3cf0DEc22": 28029089504825716,
    "0x4200000000000000000000000000000000000006": 30245221689046677
}


In [29]:
nft_pos = get_nft_positions_details('0x03a520b32C04BF3bEEf7BEb72E919cf822Ed34f1', w3, 91470)
print(json.dumps(nft_pos, indent=4))

{
    "nonce": 0,
    "operator": "0x0000000000000000000000000000000000000000",
    "token0": "0x50c5725949A6F0c72E6C4a641F24049A917DB0Cb",
    "token1": "0xd9aAEc86B65D86f6A7B5B1b0c42FFA531710b6CA",
    "fee": 100,
    "tickLower": -276336,
    "tickUpper": -276315,
    "liquidity": 149899570935539233,
    "feeGrowthInside0LastX128": 21550026498249782294602674559923957808042,
    "feeGrowthInside1LastX128": 22702457499150940817188780766,
    "tokensOwed0": 0,
    "tokensOwed1": 0
}
